# Graphes avec Dijkstra et A*

Présenté par Hugo THOLLON, Nicolas HO, Naria SAVARY

## Introduction

A mesure que la technologie a évolué, et avec elle la mobilité de l'humanité, puis des individus. Cette mobilité accrue a amené de nouveaux problèmes. Là où, à une époque, monsieur et madame tout le monde n'allaient pas plus loin que la boulangerie du village, avec l'accès à la mobilité, notamment depuis la généralisation de l'automobile, les réseaux routiers se sont complexifiés, créant un besoin de trouver des chemins de plus en plus complexe, de plus en plus rapidement. Ainsi, les techniques qui servaient à choisir les routes ont dû évoluer, et avec l'ère de l'informatique, ces techniques sont devenus des algorithmes qui ont dû être optimisés pour satisfaire les exigences croissantes du réseau routier et de ses usagers.

Dans ce rapport nous allons présenter certains de ces algorithmes en commençant par l'algorithme de Dijkstra puis une de ses dérivée appelée A*. Nous finirons ce rapport en présentant HPA* (Hierarchical Pathfinding A*) une amélioration possible de A*.  

## Installation

Avant de commencer ce rapport, nous vous invitons a créer un environnement virtuel Python et l'activer car nous aurons besoin plus tard d'importer des librairies.

```bash
python3 -m venv .venv
source .venv/bin/activate  # Linux/macOS
.venv\Scripts\activate     # Windows
```

## Dijkstra

### Présentation

_D'après Wikipedia_, l'algorithme de Dijkstra permet de résoudre le problème du plus court chemin. Ce problème a plusieurs variantes. La plus simple est la suivante : étant donné un graphe non-orienté, dont les arêtes sont munies de poids, et deux sommets de ce graphe, trouver un chemin entre les deux sommets dans le graphe, de poids minimum. L'algorithme de Dijkstra permet de résoudre un problème plus général : le graphe peut être orienté, et l'on peut désigner un unique sommet, et demander d'avoir la liste des plus courts chemins pour tous les autres nœuds du graphe.

Dans un graphe, le poids d’une arête représente la “coût” associé au passage entre deux sommets. Ce coût peut modéliser de nombreuses choses selon le contexte du problème : 
- une distance géographique, 
- un temps de trajet, 
- une consommation d’énergie, 
- un prix, 
- une difficulté.

L’algorithme de Dijkstra cherche alors à minimiser la somme de ces coûts. Choisir ce que représente le poids revient à définir ce que l’on veut optimiser dans le chemin trouvé.

Nous allons utiliser le terme **g(n)** pour parler de la **somme des poids des arêtes entre le nœud de départ D et n un nœud du graphe**. Il peut exister plusieurs chemins entre D et n. g(n) fait réference au chemin avec le plus petit poids découvert pour l'instant.

Dans l'algorithme de Dijkstra, les poids sont forcément positifs. Pour des poids négatifs il est préferable d'utiliser l'[algorithme de Bellman-Ford](https://fr.wikipedia.org/wiki/Algorithme_de_Bellman-Ford) dont nous ne parlerons pas ici.  

Dans nos implémentations et pour cette présentation, nous considèrerons que les **graphes sont non-orientés** et que les **poids** représentent la **distance en mètre** entre 2 nœuds.

### Explication avec un exemple

#### Théorie

Dans cette présentation, nous allons utiliser le graphe suivant pour expliquer le fonctionnement de Dijkstra et A* : 

<center>

<img src="./assets/graph_example_without_weight.png" alt="image du graphe d'exemple" style="background-color:white; max-width:300px;" />

_**figure 1:** graphe d'exemple_

</center>

Dans ce graphe, nous avons représenté le poids de chaque arête. Au fur et à mesure que nous compléterons le graphe nous remplacerons **00** par la valeur de g(n).

Il est possible d'exécuter l'algorithme de Dijkstra avec un tableau :  
Chaque étape correspond à une ligne.  
Une **ligne** donne les distances des sommets depuis le sommet de départ (càd le **poids des nœuds**).  
Une **colonne** donne l'évolution de g(n) d'un nœud n.  

**A chaque étape** (Sur chaque ligne), le nœud avec le plus petit g(n) est **choisi** et souligné.  
**A l'étape suivante** le g(n) des nœuds adjacent (___na___) au nœud choisi (___nc___) sont recalculés comme suit :

<center>

___g(na)___ = ___g(nc)___ + ___Poids Arête entre na et nc___

</center>

Dans le tableau, si le nouveau ___g(na)___ est inférieur à la valeur de **g(na)** déjà présente, l'ancien g(na) est remplacé par le nouveau.

Une fois un nœud n choisi, il ne peut plus être choisi. Pour rendre le tableau plus lisible, son g(n) dans le tableau est remplacé par **-**.

Si g(n) n'est pas encore connu pour un nœud n, g(n) est noté ∞ (infini).

L'algorithme s'arrête quand le nœud choisi est le nœud d'arrivée ou si tous les nœuds ont déjà été explorés.

#### Pratique

Pour montrer le fonctionnement de l'algorithme vous trouverez en dessous un GIF déroulant l'algorithme étape par étape avec le graphe présenté dans la figure 1 et un tableau qui s'actualise comme décrit dans la partie théorie.

Le code couleur du GIF est le suivant : 
- <span style="color:#CCFF99">Vert clair</span> : Départ.
- <span style="color:#66B2FF">Bleu</span> : Nœud découvert (càd que l'algorithme peut choisir) qui n'ont pas encore été choisis.
- <span style="color:#FF9999">Rouge rosé</span>/<span style="color:#E5CCFF">Rose-violet</span> : Arrivée avant et après avoir été découverte.
- <span style="color:#FFFF00">Jaune</span> : Le nœud choisi à une étape N.
- <span style="color:#00FF00">Vert</span> : Chemin final le plus court entre A et J.

<center>

<img src="./assets/dijkstra_steps.gif" alt="GIF expliquant l'algorithme de Dijkstra en plusieurs étapes" style="background-color:white; max-width:500px;" />

_**figure 2:** GIF expliquant l'algorithme de Dijkstra en plusieurs étapes_

</center>

Pour visualiser chaque étapes individuellement (sans le GIF) vous pouvez aller dans `./assets/dijkstra_steps/` et visualiser chaque image. 


### Le problème de Dijkstra

Bien que l'algorithme de Dijkstra soit très pratique pour trouver le chemin le plus court, il possède plusieurs limitations, la plus importante étant sa vitesse d'execution.  
En effet, l'algorithme va explorer toutes les directions pour trouver le chemin plus court, cela va entraîner l’exploration de nombreux chemins inutiles (ex: qui s'éloignent de la cible) avant d’atteindre l’arrivée.  
Explorer ces nœuds inutiles prend de la puissance du calcul et fait donc perdre du temps. On est alors en droit de se poser la question suivante :  
<center>

**Y a-t-il une solution pour éviter au maximum ces nœuds inutiles ?**

</center>

La réponse est oui et cette solution est l'**algorithme A\***.

## A*

### Présentation et explication

L'algorithme A* (prononcé _A étoile_ ou _A star_) est une amélioration de l'algorithme de Dijkstra. Les deux algorithmes s'éxecutent exactement de la même manière mais A* va introduire une valeur **heuristique** au calcul. 

La valeur **heuristique**, notée **h(n)**, est une **estimation du coût restant** pour atteindre l’objectif depuis un nœud n donné. Elle va permettre de guider et "orienter" la recherche en indiquant quels nœuds semblent les plus prometteurs à explorer. Une valeur heuristique ne doit pas dépasser le coût réel (càd le coût entre le nœud de départ et d'arrivé) sinon certain chemins potentiellement viables pourraient être ignorés.

On peut comparer cette valeur heuristique à une **odeur** qu'un animal va suivre pour lui indiquer la direction la plus probable de sa nourriture.  
On peut aussi faire une comparaison avec une **pente** sur laquelle de l'eau s'écoule en suivant la pente la plus favorable pour descendre.

Concrètement, l'algorithme de Dijkstra va utiliser les valeurs de **g(n)** (somme des poids des arêtes entre le nœud n et le nœud de départ) pour trouver le chemin le plus court tandis que A* va utiliser les valeurs de **f(n)** où : 

$$ {\displaystyle f(n)=g(n)+h(n)} $$

Exécuter l'algorithme de Dijkstra reviens à exécuter l'algorithme A* avec une heuristique h() toujours égale à 0.

Il est à noter que l'algorithme A* ne retourne pas forcement LE chemin le plus court (a cause des heuristiques qui influencent le résultat). Le chemin retourné est tout de même, la plupart du temps, suffisamment acceptable surtout si l'on prend en compte le temps d'execution réduit (l'algorithme A* met en moyenne 2 fois moins de temps à s'executer que l'algorithme de Dijkstra).

Bien choisir la valeur retournée par l'heuristique est essentiel car c'est ce qui déterminera à quel point l'algorithme A* est efficace. Nous en avons fait les frais quand, lors de nos experimentations, nous avions converti des distances en kilomètres au lieu de mètres : tous nos calculs étant en mètres nous avions des distances comme **g(n)=3121** avec des heuristiques qui auraient dû être **h(n)=1357** mais se sont transformées en **h(n)=1.357**, ce qui les rendait quasiment inutiles. 

Comme une image vaut milles mots vous pouvez retrouver ci-dessous une comparaison côte à côte des algorithmes de Dijkstra et A* avec en <span style="color:green">vert</span> le chemin final et en <span style="color:magenta">magenta</span> les chemins explorés par les algorithmes.  
Cette comparaison a été réalisée avec le code Python que vous retrouverez plus bas.  
Les deux algorithmes ont été exécuter l'un après l'autre sur la carte de Toulouse avec Basso-cambo comme point de départ et Jolimont comme point d'arrivée. L'heuristique utilisé est la distance à vol d'oiseau entre le nœud n et le nœud d'arrivée.

<center>

<img src="./assets/comparison_dijkstra_astar.png" alt="image comparaison des algorithmes Dijkstra et A*" style="background-color:white; max-width:600px;" />

_**figure 3:** Comparaison des algorithmes de Dijkstra (à gauche) et A* (à droite)_

</center>

Nous pouvons très clairement voir que l'algorithme A* a eu besoin d'explorer beaucoup moins de chemin que l'algorithme de Dijkstra pour arriver au résultat. Nous pouvons aussi retrouver cette idée d'eau qui coule d'une pente pour l'algorithme A* et d'eau qui se répand pour l'algorithme de Dijkstra.

### Pour résumer

L'algorithme A* est une amélioration de l'algorithme de Dijkstra qui le rend beaucoup plus rapide en ajoutant une valeur heuristique au calculs.

La valeur heuristique peut varier d'une implémentation à l'autre et selon le besoin. La valeur heuristique ne doit pas dépasser le coût réel final pour ne pas exclure des solutions potentiellement viables.

L'algorithme A* utilise la somme des poids des arêtes entre un nœud n et le nœud de départ, noté g(n), additionné à une valeur heuristique associée au nœud n, noté h(n), pour calculer le coût du nœud n, noté f(n). A chaque étape l'algorithme A* selectionnera le nœud n avec le plus petit f(n).

L'algorithme A* ne garantit pas LE meilleur résultat mais permet tout de même d'obtenir un résultat proche du meilleur.

### A* sur un graphe

TODO

### A* en code

Dans cette partie de la présentation nous allons implémenter l'algorithme A* en utilisant la librairie python `osmnx` pour obtenir une carte d'une zone donnée avec OpenStreetMap, la librairie `matplotlib` pour afficher cette carte et la librairie `haversine` pour obtenir la distance à vol d'oiseau entre 2 nœuds à l'aide de leur latitude et longitude.

_La librairie haversine fournit simplement une implémentation de la **formule de haversine** qui permet de déterminer la distance entre deux points d'une sphère (ici la Terre), à partir de leurs longitudes et latitudes._


In [ ]:
# Installation des dépendances
%pip install pandas pandas-stubs
%pip install haversine
%pip install osmnx
%pip install scikit-learn
%pip install matplotlib
%pip install types-networkx

Dans le code qui suit, certaines constantes sont commentées car non utilisées. Vous pouvez les décommenter pour changer les données utilisées par le programme.

In [ ]:
# Import des librairies et définition des constantes
import os.path
import osmnx
import matplotlib.pyplot as plt
import heapq
from haversine import haversine, Unit


# Le nom d'une ville/région/pays comme écrit dans Nominatim (outil pour visualiser des lieux avec OpenStreetMap)
# place_name = "Toulouse, Haute-Garonne, Occitania, Metropolitan France, France"
place_name = "Haute-Garonne, Occitania, Metropolitan France, France"
# place_name = "Occitania, Metropolitan France, France"
# place_name = "Metropolitan France"


# Coordonnées de différents nœud en France
START_COORD = (1.3922743, 43.5699769)  # (lon/lat) N7681108802 Basso-cambo
# END_COORD = (1.4629966, 43.6143038)  # (lon/lat) N305142882 - Jolimont
END_COORD = (0.7247218, 43.1077682)  # (lon/lat) N26691893 - Saint-Gaudens
# END_COORD = (4.4041170, 43.8692150)  # (lon/lat) N495652597 - Courbessac (Nîmes)
# END_COORD = (48.8532677, 2.3478864)  # (lat/lon) N11111197772 - Notre-Dame de Paris


# Défini combien d'étapes passent avant que la carte de matplotlib ne soit mise à jour avec les nouveaux chemins choisi
UPDATE_MAP_EVERY_N_STEP = 100

Chargement de la carte depuis OpenStreetMap avec osmnx. Placement des coordonnées des nœuds dans un dictionnaire. Initialisation de la carte avec matplotlib.

In [ ]:
# Transform this name in a filepath with the extension graphml
filePath = "./cache_data/" + place_name.replace(", ", "_") + ".graphml"
# If there is a file, load it, it's much faster that downloading the entire graph every time
if os.path.isfile(filePath):
    print("Loading graph from " + filePath)
    G = osmnx.load_graphml(filePath)
    print("Graph loaded")
else:
    # If it's the first time, download the graph from Nominatim using the place name.
    # Download only the roads (network_type="drive").
    print("Download graph of: " + place_name)
    G = osmnx.graph_from_place(place_name, network_type="drive")
    print("Graph downloaded")
    print("Saving the downloaded graph in " + filePath)
    osmnx.save_graphml(G, filePath)
    print("Graph saved")

# Simplify the graph: cleans up nodes by collapsing intermediate nodes (ex: nodes that are not
# intersections, just points used to draw a curved road) into one.
# Graph simplification is already done in osmnx.graph_from_place by default.
try:
    G = osmnx.simplify_graph(G)
except:  # noqa: E722  <-- Remove a lint error
    print("Graph cannot be simplified again.")
    

orig_node = osmnx.distance.nearest_nodes(G, X=START_COORD[0], Y=START_COORD[1])
dest_node = osmnx.distance.nearest_nodes(G, X=END_COORD[0], Y=END_COORD[1])

# Precompute node coordinates, x=lon, y=lat
node_positions = {node: (data["x"], data["y"]) for node, data in G.nodes(data=True)}
# Without data=True, G.nodes() returns only the node ID

# Draw the base map
_, ax = osmnx.plot_graph(
    G, show=False, close=False, node_size=0, edge_color="#CCCCCC", bgcolor="white"
)

# Highlight start and goal nodes
ax.scatter(*node_positions[orig_node], c="green", s=80, zorder=5, label="Start")
ax.scatter(*node_positions[dest_node], c="red", s=80, zorder=5, label="Goal")

plt.legend()
# Interactive mode on
plt.ion()
plt.show()


### Le problème de A*